In [2]:
import os
import tensorflow as tf
import pandas as pd

tf.__version__


'2.13.0'

In [3]:
main_dir = r"F:\linux_ubuntu_codes\codes\nlp_lecture"

In [4]:
train_dir = os.path.join(main_dir + r"\train.csv")
test_dir = os.path.join(main_dir + r"\test.csv")

'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/train.csv'

In [5]:
train_dir

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# Make a train_df
train_df = pd.read_csv(train_dir)
test_df = pd.read_csv(test_dir)
train_df.head()

,id,keyword,location,text,target
3228,4632,emergency%20services,"Sydney, New South Wales",Goulburn man Henry Van Bilsen missing: Emergen...,1
3706,5271,fear,NaN,The things we fear most in organizations--fluc...,0
6957,9982,tsunami,Land Of The Kings,@tsunami_esh ?? hey Esh,0
2887,4149,drown,NaN,@POTUS you until you drown by water entering t...,0
7464,10680,wounds,"cody, austin follows ?*?",Crawling in my skin\nThese wounds they will no...,1


In [7]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state = 1)
train_df_shuffled.head()

(7613, 5)

In [8]:
train_df_shuffled.shape

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# How many examples of each class?
train_df['target'].value_counts()

(7613, 3263)

In [10]:
# How many total samples?
len(train_df), len(test_df)

Target:0 (not a real disaster)
Text: 
The best thing about this is it drowned out the call from the guy angry cause he hadn't gotten a tracking number... http://t.co/QYu8grOrQ1
-----

Target:0 (not a real disaster)
Text: 
And when those waves shall ripple collide it's on the tide of YOUR LOVE I will survive. #love @LesleyChappelle
-----

Target:1 (real disaster)
Text: 
@fewmoretweets all lives matter. Just not a fan of burning down buildings and stealing from your neighbors to 'protest'
-----

Target:0 (not a real disaster)
Text: 
The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'
-----

Target:0 (not a real disaster)
Text: 
@Kiwi_Karyn Check out what's in my parking lot!! He said that until last year it was an ambulance in St Johns. http://t.co/hPvOdUD7iP
-----


In [10]:
# Visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index: random_index +5].itertuples():
    _, text, target = row
    if target > 0:
        print(f"Target:{target}", "(real disaster)")
    else:
        print(f"Target:{target}", "(not a real disaster)")
    
    print(f"Text: \n{text}")
    print("-----\n")

Target:0 (not a real disaster)
Text: 
.@bigperm28 was drafted by the @Avalanche in 2005 (rd. 4 #124) overall. Played last season in @UtahGrizz. http://t.co/gPGTAfMKt0
-----

Target:0 (not a real disaster)
Text: 
self harm// I'm so angry please tag your scars on tumblr jesus christ i dont wanna get triggered
-----

Target:1 (real disaster)
Text: 
Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Marians a... http://t.co/BHZr9UgUs2
-----

Target:0 (not a real disaster)
Text: 
Human history becomes more and more a race between education and catastrophe.
-----

Target:1 (real disaster)
Text: 
Wreckage 'conclusively confirmed' as from MH370: Malaysia PM http://t.co/Rp2DxFKHDQ | https://t.co/akmIHLRIs1
-----


### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df['text'],
                                                                            train_df['target'],
                                                                            test_size = 0.1,
                                                                            random_state = 42)

(7613,)

In [13]:
train_df['text'].shape

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [14]:
train_df["text"]

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [15]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [16]:
train_df['text'].to_numpy()

(6851, 762, 6851, 762)

In [16]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

### Converting text into numbers: Text Vectorization also known as Tokenization

In [18]:
# Get the total words
total_words = 0
for i in train_sentences:
    total_words += len(i.split())

total_words

(6851,)

In [19]:
train_sentences.shape

15

In [20]:
# Get the average words per sentence or line
avg_words_per_sentence = round(total_words/len(train_sentences))
avg_words_per_sentence

In [21]:
# Normally, total number of words is equal or less than the max vocabulary length
max_vocab_length = 10000

# max length of a statement is equal or more than the average word per sentence or line
max_length = 15

In [22]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

There's a flood in my street! Sentence length is 29.


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[282,   3, 206,   4,  13, 674,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [23]:
# Create a sample sentence and tokenize it
samp_sentence = "There's a flood in my street!"
print(samp_sentence, f"Sentence length is {len(samp_sentence)}.")
text_vectorizer([samp_sentence])
#len(tf.squeeze(text_vectorizer([samp_sentence])))

Original text: 

 Metro acting chief Jack Requa says train that derailed was a six-car train with 1000 and 2000 series rail cars. #wmata 

Vectorized text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1383, 4159, 1888, 2081, 9152,  273,  118,   16,  606,   23,    3,
        8433,  118,   14, 1577]], dtype=int64)>

In [24]:
# Choose random sentence from the training dataset and tokenize it
rand_train_sentence = random.choice(train_sentences)
print(f"Original text: \n\n {rand_train_sentence} \n\nVectorized text:")
text_vectorizer([rand_train_sentence])

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'are',
 'be',
 'was',
 'have',
 'like',
 'as',
 'me',
 'but',
 'up',
 'just',
 'so',
 'im',
 'not',
 'amp',
 'your',
 'out',
 'all',
 'after',
 'its',
 'has',
 'no',
 'will',
 'an',
 'fire',
 'when',
 'if',
 'we',
 'get',
 'now',
 'more',
 'via',
 'new',
 'about',
 'what',
 'people',
 'or',
 'news',
 'he',
 'they',
 'over',
 'one',
 'been',
 'how',
 'dont',
 'who',
 'video',
 'into',
 'were',
 'do',
 'us',
 'can',
 'emergency',
 '2',
 'disaster',
 'there',
 'her',
 'some',
 'than',
 'still',
 'would',
 'his',
 'police',
 'crash',
 'burning',
 'suicide',
 'body',
 'california',
 'back',
 'time',
 'buildings',
 'had',
 'why',
 'off',
 'them',
 'got',
 'man',
 'see',
 'storm',
 'know',
 'going',
 'cant',
 'world',
 'first',
 'day',
 'rt',
 'nuclear',
 'love',
 'youtube',
 'our',
 'attack',
 'go',
 'fires',
 'two',
 'their',
 'bomb',

In [25]:
# Get unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab

10000

In [26]:
len(words_in_vocab)

['', '[UNK]', 'the', 'a', 'in']

In [27]:
# Top 5 words in words_in_vocab
words_in_vocab[:5]

['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']

In [27]:
# Least 5 words in words_in_vocab
words_in_vocab[-5:]

['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']

### Embedding layer

To make our embedding, we are going to use TensorFlow’s embedding layer: https://www.tensorflow.org/api/docs/python/tf/keras/layers/

The parameters are below mostly used.
* input_dim = size of the vocabulary
* output_dim = size of the output embedding vector.  For example, a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

In [29]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 64,
                             embeddings_initializer = "uniform",
                             input_length = max_length)

Original text: Soul food sound so bomb right now '

Vectorized text: [[1048  327  564   31  115  175   49    0    0    0    0    0    0    0
     0]]

Vectorized text shape: (1, 15)
Embbedded text: [[[ 1.92673914e-02  1.03229284e-02 -2.99850702e-02 -3.83953825e-02
    8.22282955e-03 -4.90010865e-02 -4.65819128e-02  3.74317169e-03
   -4.54433225e-02  4.54574712e-02 -2.55725142e-02 -4.02441509e-02
    1.60885192e-02  2.69680358e-02 -9.90509987e-03  6.56571239e-03
   -4.66850065e-02 -2.91888602e-02 -6.04242086e-06 -4.77750190e-02
   -2.63387207e-02  3.77299078e-02  1.86421983e-02  4.85537089e-02
   -9.55302641e-03 -4.17189598e-02  3.59614380e-02  4.19400968e-02
    1.69054903e-02  4.75232266e-02 -4.13823016e-02 -4.27360460e-03
   -4.80304025e-02  3.66009586e-02  1.93724297e-02 -3.08783781e-02
    1.41225196e-02  3.63617428e-02 -1.86781771e-02  2.01191790e-02
   -1.19296908e-02  5.32950088e-03  5.80988824e-04  1.84386633e-02
   -2.60384567e-02  3.68697904e-02 -2.10559852e-02  2.96113528e-0

In [29]:
# Get a random sentence from the training set and return Original text, Vectorized text, Embedded text
rand_sentence = random.choice(train_sentences)
print(f"Original text: {rand_sentence}\n")
print(f"Vectorized text: {text_vectorizer([rand_sentence])}\n")
print(f"Vectorized text shape: {text_vectorizer([rand_sentence]).shape}")
print(f"Embbedded text: {embedding(text_vectorizer([rand_sentence]))}")
print(f"Embedded text shape: {embedding(text_vectorizer([rand_sentence])).shape}")

Original text: THE NATIONAL WEATHER SERVICE IN LITTLE ROCK HAS ISSUED A * SEVERE THUNDERSTORM WARNING FOR... VAN BUREN COUNTY IN Û_ http://t.co/KJsvW06GBV

Vectorized text: [[   2  433  242  319    4  245  979   40  897    3  250  271  269   10
  1934]]

Vectorized text shape: (1, 15)
Embbedded text: [[[-3.02148946e-02  4.40153964e-02  1.71712376e-02 -3.49548832e-02
    2.89961956e-02 -3.67722027e-02  1.05644949e-02  2.94112600e-02
    5.02588600e-03  2.70172469e-02 -1.46191008e-02 -2.74715908e-02
   -8.73107836e-03  4.29104641e-03  4.27779295e-02 -2.58139372e-02
   -1.73365474e-02  4.93229739e-02  3.45834978e-02 -2.74662375e-02
   -4.56687696e-02  4.91838790e-02  4.43506353e-02 -3.70482318e-02
   -3.71547565e-02 -1.31838098e-02  2.34005339e-02  9.27425921e-04
   -3.86318676e-02 -1.24239326e-02  1.31485127e-02  2.25189440e-02
   -4.91767190e-02 -4.40330505e-02  9.02072340e-03 -4.54564467e-02
    1.14731863e-03 -3.22664604e-02  9.95707512e-03  1.26960017e-02
   -2.31000427e-02  4.52465

### Model_2: Long Short Term Memory (LSTM) Model

LSTM = Long short term memory (One of the most popular LSTM cells)

Our structure of a RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/Dense) -> Output (label probability)
```

In [31]:
# Create a LTSM Model

from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences = True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name ="model_2_LSTM")

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 15, 64)            640000    
                                                                 
 lstm (LSTM)                 (None, 15, 64)            33024     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                      

In [32]:
model_2.summary()

In [33]:
# Compile the model
model_2.compile(loss = "binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

Epoch 1/5
215/215 [==============================] - 5s 11ms/step - loss: 0.5144 - accuracy: 0.7473 - val_loss: 0.4737 - val_accuracy: 0.7900
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3185 - accuracy: 0.8647 - val_loss: 0.5188 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2224 - accuracy: 0.9178 - val_loss: 0.5984 - val_accuracy: 0.7297
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1663 - accuracy: 0.9404 - val_loss: 0.6175 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1332 - accuracy: 0.9559 - val_loss: 0.6881 - val_accuracy: 0.7310


In [34]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels))

24/24 [==============================] - 0s 2ms/step


array([[0.03208097],
       [0.20279762],
       [0.07481588],
       [0.04135088],
       [0.9811979 ],
       [0.10899878],
       [0.00713235],
       [0.05714665],
       [0.13276812],
       [0.9987485 ],
       [0.9664942 ],
       [0.6166183 ],
       [0.04905394],
       [0.5674198 ],
       [0.18249074],
       [0.48025864],
       [0.93565536],
       [0.01874873],
       [0.9999223 ],
       [0.01149162],
       [0.45840392],
       [0.88950324],
       [0.6805498 ],
       [0.43790504],
       [0.18323156],
       [0.04545465],
       [0.9997286 ],
       [0.92811435],
       [0.999953  ],
       [0.00836265],
       [0.22284022],
       [0.01387595],
       [0.3706735 ],
       [0.8252464 ],
       [0.1753077 ],
       [0.5730591 ],
       [0.92941034],
       [0.16509645],
       [0.0139841 ],
       [0.32049307],
       [0.04819231],
       [0.9998144 ],
       [0.12789322],
       [0.8939987 ],
       [0.04045928],
       [0.07363887],
       [0.9949272 ],
       [0.190

In [35]:
# Make predictions using the model_2: LSTM Model
model_2_preds_probs = model_2.predict(val_sentences)
model_2_preds_probs

(762, 1)

In [36]:
model_2_preds_probs.shape

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1.,
       0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0.,
       0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1.,
       1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1

In [37]:
# Convert model_2_preds_probs to the val_labels'format(0,1)
model_2_preds = tf.squeeze(tf.round(model_2_preds_probs))
model_2_preds

In [38]:
# Evaluate the model by comparing the model_2_preds to val_labels
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_2_accuracy_score = accuracy_score(val_labels, model_2_preds)
model_2_precision, model_2_recall, model_2_fscore, _ =precision_recall_fscore_support(val_labels,
                                                                                      model_2_preds,
                                                                                      average = "weighted")

0.730888110481174

In [39]:
model_2_precision

{'model_2_accuracy_score': 0.7309711286089239,
 'model_2_precision': 0.730888110481174,
 'model_2_recall': {0.7309711286089239},
 'model_2_f1score': {0.730928355705545}}

In [44]:
dict_result = {"model_2_accuracy_score": model_2_accuracy_score,
               "model_2_precision": model_2_precision,
               "model_2_recall": {model_2_recall},
               "model_2_f1score": {model_2_fscore}}
dict_result

{'model_2_accuracy_score': 0.7388451443569554,
 'model_2_precision': 0.7379198684322771,
 'model_2_recall': {0.7388451443569554},
 'model_2_f1score': {0.7380105609849744}}